In [1]:
using JuMP
using ConditionalJuMP
using MIPVerify
using MathProgBase
using LinearAlgebra

# You can use your solver of choice; I'm using Gurobi for my testing.
using Gurobi

# Setup
The `OptimizationProblem` struct and `get_optimization_problem` function are all you need to get started.

In [2]:
struct OptimizationProblem{T<:Union{JuMP.Variable,JuMP.AffExpr}}
    model::Model
    input_variable::Array{Variable}
    output_variable::Array{<:T}
end

In [3]:
"""
:param input_size: size of input to neural network
:param lower_bounds: element-wise lower bounds to input
:param upper_bounds: element-wise upper bounds to input
:param set_additional_input_constraints: Function that accepts the 
    input variable (in the form of Array{<:Union{JuMP.Variable,JuMP.AffExpr}}) 
    and sets any additional constraints. Example function `f` that sets the
    first element of the input to 

```
function f(v_in)
    m = MIPVerify.getmodel(v_in)
    @constraint(m, v_in[1] <= 1)
end
```

:returns: Optimization problem with model, input variables, and output variables.
    Please set all constraints on input variables via `set_additional_input_constraints`
    so that the information is available when propagating bounds forward through
    the network.

"""
function get_optimization_problem(
    input_size::Tuple{Int}, 
    nn::NeuralNet, 
    solver::MathProgBase.SolverInterface.AbstractMathProgSolver;
    lower_bounds::AbstractArray{<:Real} = zeros(input_size), 
    upper_bounds::AbstractArray{<:Real} = ones(input_size),
    set_additional_input_constraints::Function = _ -> nothing
)::OptimizationProblem
    @assert(
        size(lower_bounds) == input_size,
        "Lower bounds must match input size $input_size"
    )
    @assert(
        size(upper_bounds) == input_size,
        "Upper bounds must match input size $input_size"
    )
    @assert(
        all(lower_bounds .<= upper_bounds),
        "Upper bounds must be element-wise at least the value of the lower bounds"
    )    
    
    m = Model()
    JuMP.setsolver(m, solver)
    input_range = CartesianIndices(input_size)

    # v_in is the variable representing the actual range of input values
    v_in = map(
        i -> @variable(m, lowerbound = lower_bounds[i], upperbound = upper_bounds[i]), 
        CartesianIndices(input_size)
    )
    
    # these input constraints need to be set before we feed the bounds
    # forward through the network via the call nn(v_in)
    set_additional_input_constraints(v_in)

    return OptimizationProblem(m, v_in, nn(v_in))
end


get_optimization_problem

# Example 1: Identity Network with Simple Element-wise Bounds

In the example below, the input and output variables are identical, and the only constraints are the element-wise constraints on the input variables.

In [4]:
# single layer identity network 
nn_id = Sequential([Linear(Matrix(1.0I, 4, 4), zeros(4,))], "identity")

sequential net identity
  (1) Linear(4 -> 4)


In [5]:
p1 = get_optimization_problem(
    (4,), 
    nn_id, 
    GurobiSolver(),
    lower_bounds=[1, 2, 3, 4],
    upper_bounds=[5, 6, 7, 8],
)

OptimizationProblem{JuMP.GenericAffExpr{Float64,Variable}}(Feasibility problem with:
 * 0 linear constraints
 * 4 variables
Solver is Gurobi, Variable[__anon__, __anon__, __anon__, __anon__], JuMP.GenericAffExpr{Float64,Variable}[__anon__, __anon__, __anon__, __anon__])

When we minimize the sum of the output variables, the objective is 10, with the input variables taking the value of the lower bound.

In [17]:
@objective(p1.model, Min, sum(p1.output_variable))
solve(p1.model)
println("\nObjective value: $(getobjectivevalue(p1.model)), input variables: $(getvalue(p1.input_variable))")

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 0 rows, 4 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.000000000e+01

Objective value: 10.0, input variables: [1.0, 2.0, 3.0, 4.0]


When we maximize the sum of the output variables, the objective is 26, with the input variables taking the value of the upper bound.

In [18]:
@objective(p1.model, Max, sum(p1.output_variable))
solve(p1.model)
println("\nObjective value: $(getobjectivevalue(p1.model)), input variables: $(getvalue(p1.input_variable))")

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 0 rows, 4 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  2.600000000e+01

Objective value: 26.0, input variables: [5.0, 6.0, 7.0, 8.0]


Here's a slightly more complex linear objective

In [23]:
o = p1.output_variable
@objective(p1.model, Max, o[1]-o[2]+o[3]-o[4])
solve(p1.model)
println("\nObjective value: $(getobjectivevalue(p1.model)), input variables: $(getvalue(p1.input_variable))")

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 0 rows, 4 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  6.000000000e+00

Objective value: 6.0, input variables: [5.0, 2.0, 7.0, 4.0]


# Example 2: Simple One-Layer Network with Slightly More Complex Constraints

In [30]:
# the convention for the `Linear` layer is a bit confusing
# I'm planning to change it, but you can refer to the docs for now
matrix = [
    1 1 1
    1 1 1
    1 1 1
    1 1 1
]

bias = [0, -2, -4]

nn_simple = Sequential([
    Linear(matrix, bias),
    ReLU(lp)
], "test")

sequential net test
  (1) Linear(4 -> 3)
  (2) ReLU()


In [31]:
function set_constraints(v_in)
    m = MIPVerify.getmodel(v_in)
    @constraint(m, v_in[1]-v_in[2]<=0.2)
    @constraint(m, v_in[1]-v_in[2]>=-0.2)
end

set_constraints (generic function with 1 method)

The output you see below are the sub-problems that are being solved as we determine the lower and upper bounds for each input to the ReLU. The type of problem we solve is controlled [here](https://github.com/vtjeng/MIPVerify.jl/blob/b6a585dab860b64e15717d5c9219b705b62fea0c/src/MIPVerify.jl#L18) and can be set on a per-layer basis; the [default is to use `mip`](https://github.com/vtjeng/MIPVerify.jl/blob/master/src/MIPVerify.jl#L20)

In [32]:
p2 = get_optimization_problem(
    (4,), 
    nn_simple, 
    GurobiSolver(),
    set_additional_input_constraints = set_constraints,
)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 2 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xf475c4a2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 2e-01]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  4.000000000e+00
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 2 rows, 4 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 2e-01]
Iteration 

OptimizationProblem{JuMP.GenericAffExpr{Float64,Variable}}(Minimization problem with:
 * 6 linear constraints
 * 6 variables: 1 binary
Solver is Gurobi, Variable[__anon__, __anon__, __anon__, __anon__], JuMP.GenericAffExpr{Float64,Variable}[__anon__ + __anon__ + __anon__ + __anon__, __anon__, 0])

In [27]:
o = p2.output_variable
@objective(p2.model, Max, o[1]-o[2])
solve(p2.model)
println(getobjectivevalue(p2.model))
println("\nObjective value: $(getobjectivevalue(p2.model)), input variables: $(getvalue(p2.input_variable))")

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 6 rows, 6 columns and 18 nonzeros
Model fingerprint: 0x396fcddd
Variable types: 5 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+00]
  RHS range        [2e-01, 2e+00]

User MIP start produced solution with objective -0 (0.00s)
Loaded user MIP start with objective -0

Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%
2.0

Objective value: 2.0, input variables: [0.5999999999999994, 0.39999999999999947, 1.0, 1.1102230246251565e-15]
